# Analysis on a Preliminary Merge

Import libraries

In [9]:
import pandas as pd
import LatLongtoBlock
import pygeocoder
#import address_to_lat_long as atll

### Read in desired .csv files and glimpse structure

In [44]:
crime = pd.read_csv('crime_ll.csv').drop(crime.columns[0], axis=1)
crime.head()

,Unnamed: 0,Offense,IncidentID,BlockNumber,StreetName,Agency,DateReported,HourReported,Address,Latitude,Longitude
0,0,Burglary,201000073238,1700,ALLIED LN,CPD,2017-09-22T16:00:00.000Z,1200,"1700 ALLIED LN, Charlottesville VA",38.041381,-78.479806
1,1,Drug/Narcotics Violation,201300002276,1200,LANDONIA CIR,CPD,2013-12-13T16:03:00.000Z,1103,"1200 LANDONIA CIR, Charlottesville VA",38.036315,-78.462432
2,2,Drug/Narcotics Violation,201300002481,500,WATER ST,CPD,2014-01-02T19:00:00.000Z,1400,"500 WATER ST, Charlottesville VA",38.029081,-78.478864
3,3,Drug/Narcotics Violation,201300003848,100,DICE ST,CPD,2014-02-18T23:30:00.000Z,1830,"100 DICE ST, Charlottesville VA",38.027426,-78.483989
4,4,Drug/Narcotics Violation,201300004323,1800,HYDRAULIC RD,CPD,2013-12-27T19:00:00.000Z,1400,"1800 HYDRAULIC RD, Charlottesville VA",38.060175,-78.490386


In [45]:
busstops = pd.read_csv('./data/cat_bus_stop_points.csv')
busstops.head()

,X,Y,OBJECTID,stop_id,stop_code,stop_name,stop_lat,stop_lon
0,-78.507706,38.036545,2916,1,10472,Emmet St at Central Grounds Garage,38.036545,-78.507706
1,-78.507065,38.038326,2917,2,10660,Emmet St at Snyder Tennis Courts,38.038326,-78.507065
2,-78.516357,38.029072,2918,3,10687,Alderman Road at Stadium Rd northbound,38.029072,-78.516357
3,-78.480141,38.029556,2919,4,11015,East Water St at 3rd St SE,38.029556,-78.480141
4,-78.484268,38.030975,2920,5,11036,West Water St at Omni,38.030975,-78.484268


### Adjust data to have block numbers

Add a Block column to busstops (by converting lat/long to a block number)

In [10]:
busstops['Block'] = LatLongtoBlock.lltoblock(busstops)

Check that it went through okay

In [11]:
busstops.head()

,X,Y,OBJECTID,stop_id,stop_code,stop_name,stop_lat,stop_lon,Block
0,-78.507706,38.036545,2916,1,10472,Emmet St at Central Grounds Garage,38.036545,-78.507706,510030109032003
1,-78.507065,38.038326,2917,2,10660,Emmet St at Snyder Tennis Courts,38.038326,-78.507065,510030109032003
2,-78.516357,38.029072,2918,3,10687,Alderman Road at Stadium Rd northbound,38.029072,-78.516357,510030109031003
3,-78.480141,38.029556,2919,4,11015,East Water St at 3rd St SE,38.029556,-78.480141,515400010001043
4,-78.484268,38.030975,2920,5,11036,West Water St at Omni,38.030975,-78.484268,515400010001020


Store it

In [57]:
busstops.to_csv('./data_with_blocks/busstops_with_block.csv')

Add a block column to crime (by converting from address to lat/long to a block number)

In [27]:
'X' in crime.columns and 'Y' in crime.columns

False

In [39]:
import urllib.request
import re
def lltoblock(df):
    
    # Create empty array for storing block number
    block_list = pd.Series([])
    
    # Create opener
    opener = urllib.request.FancyURLopener({})
    
    if 'X' in df.columns and 'Y' in df.columns:
        lat = df['Y']
        lng = df['X']
    elif 'Latitude' in df.columns and 'Longitude' in df.columns:
        lat = df['Latitude']
        lng = df['Longitude']
    elif 'Meter_Lat' in df.columns and 'Meter_Long' in df.columns:
        lat = df['Meter_Lat']
        lng = df['Meter_Long']
    else:
        lat = None
        lng = None
    
    
    # Iterate over all rows of dataframe
    for i in range(len(df)):
        
        # Create url from lat/long
        url = 'https://geo.fcc.gov/api/census/block/find?latitude='\
        + str(lat[i]) + '&longitude=' + str(lng[i]) + '&showall=false&format=json'
        
        # Store webpage contents
        f = opener.open(url)
        content = f.read()
        
        # Get block number out of content string
        try:
            block = re.search(r".*FIPS\":\" *(.*?) *\",\"bbox.*", str(content)).group(1)
        except:
            block = None
        # Add current block number to our block array
        block_list[i] = block
        
    # Return the list of block numbers
    return block_list

In [40]:
crime['Block'] = LatLongtoBlock.lltoblock(crime)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: FancyURLopener style of invoking requests is deprecated. Use newer urlopen functions/methods
  if __name__ == '__main__':


Check that it went through okay

In [41]:
crime.head()

,RecordID,Offense,IncidentID,BlockNumber,StreetName,Agency,DateReported,HourReported,Address,Latitude,Longitude,Block
0,1,Burglary,201000073238,1700,ALLIED LN,CPD,2017-09-22T16:00:00.000Z,1200,"1700 ALLIED LN, Charlottesville VA",38.041381,-78.479806,515400010002017
1,2,Drug/Narcotics Violation,201300002276,1200,LANDONIA CIR,CPD,2013-12-13T16:03:00.000Z,1103,"1200 LANDONIA CIR, Charlottesville VA",38.036315,-78.462432,515400003021007
2,3,Drug/Narcotics Violation,201300002481,500,WATER ST,CPD,2014-01-02T19:00:00.000Z,1400,"500 WATER ST, Charlottesville VA",38.029081,-78.478864,515400010001062
3,4,Drug/Narcotics Violation,201300003848,100,DICE ST,CPD,2014-02-18T23:30:00.000Z,1830,"100 DICE ST, Charlottesville VA",38.027426,-78.483989,515400004011006
4,5,Drug/Narcotics Violation,201300004323,1800,HYDRAULIC RD,CPD,2013-12-27T19:00:00.000Z,1400,"1800 HYDRAULIC RD, Charlottesville VA",38.060175,-78.490386,515400008004001


In [42]:
crime.to_csv('./data_with_blocks/crime_with_block.csv')

### Merging the Data

Read in stored crime (with block) data, with block data read as a string, so we don't get unnecessary rounding/integer treatment

In [58]:
crime = pd.read_csv('./data_with_blocks/crime_with_block.csv', dtype={'Block': str})
busstops = pd.read_csv('./data_with_blocks/busstops_with_block.csv', dtype={'Block': str})

Get counts of each block's number of busstops and number of crimes

In [59]:
crime_counts = crime['Block'].value_counts()
busstop_counts = busstops['Block'].value_counts()

Check on their output

In [60]:
crime_counts.head()

515400010001009    1849
515400004012000     877
515400005013000     633
515400010001030     499
515400007002013     482
Name: Block, dtype: int64

In [61]:
busstop_counts.head()

510030107003000    5
510030109032007    5
515400003022005    5
515400004011004    4
515400004012010    4
Name: Block, dtype: int64

Merge the two

In [69]:
crime_busstops = pd.DataFrame(data=dict(Crimes=crime_counts, BusStops=busstop_counts)).fillna(0)

In [70]:
crime_busstops.head()

,Crimes,BusStops
510030101001002,1.0,0.0
510030102011019,1.0,0.0
510030102011039,1.0,0.0
510030102011052,1.0,0.0
510030102011056,12.0,0.0


Try out a test case to make sure the merge was successful

In [71]:
test_case = '515400010002023'

In [72]:
crime_busstops[crime_busstops.index == test_case]

,Crimes,BusStops
515400010002023,84.0,1.0


In [73]:
crime_counts[crime_counts.index == test_case]

515400010002023    84
Name: Block, dtype: int64

In [74]:
busstop_counts[busstop_counts.index == test_case]

515400010002023    1
Name: Block, dtype: int64

Looks good